In [35]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [30]:
df = pd.read_csv("sales_pos.csv")
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


In [31]:
df_sub = df[["job", "prod", "purchase"]].copy()
df_sub.head(2)

,job,prod,purchase
0,10,P00069042,8370
1,10,P00248942,15200


In [32]:
df_sub2 = df_sub.iloc[:, 1:]
df_sub2.head(2)

,prod,purchase
0,P00069042,8370
1,P00248942,15200


## Q1 ANSWER

In [33]:
df.shape

(550068, 11)

In [36]:
df_q1 = df[["prod", "job", "purchase"]].copy()
df_q1.head(2)

,prod,job,purchase
0,P00069042,10,8370
1,P00248942,10,15200


In [37]:
df_q1.groupby("prod")["purchase"].sum()

prod
P00000142    12837476
P00000242     3967496
P00000342     1296475
P00000442      441173
P00000542      807212
               ...   
P0099442      2870383
P0099642        83710
P0099742       991948
P0099842       737312
P0099942        78019
Name: purchase, Length: 3631, dtype: int64

In [40]:
top_prod = df_q1.groupby("prod")["purchase"].sum().idxmax()
top_prod

'P00025442'

In [41]:
df_q1_sub = df_q1.loc[df_q1["prod"] == top_prod, ]
df_q1_sub.head(2)

,prod,job,purchase
667,P00025442,17,19706
749,P00025442,7,15212


In [43]:
df_q1_sub["job"].value_counts()

4     221
7     187
0     179
17    143
1     124
12    117
14     84
2      78
20     75
16     60
10     55
6      54
15     50
3      46
11     38
5      31
13     24
19     20
18     14
9      12
8       3
Name: job, dtype: int64

In [44]:
df_q1_sub["job"].value_counts().idxmax()

4

## Q2 ANSWER

In [52]:
df_sample = df.loc[df["user"] == 1, ["prod_cat1", "prod_cat2", "prod_cat3"]]
df_sample.head(2)

,prod_cat1,prod_cat2,prod_cat3
0,3,NaN,NaN
1,1,6.0,14.0


In [53]:
len(df_sample)

35

In [54]:
df_sample.fillna(0).drop_duplicates()

,prod_cat1,prod_cat2,prod_cat3
0,3,0.0,0.0
1,1,6.0,14.0
2,12,0.0,0.0
3,12,14.0,0.0
39180,2,4.0,8.0
78144,4,8.0,9.0
78145,1,2.0,5.0
78146,3,4.0,12.0
116848,5,15.0,0.0
116849,5,0.0,0.0


In [55]:
len(df_sample.fillna(0).drop_duplicates())

21

In [56]:
df_sample = df_sample.fillna(0)
df_sample.head(2)

,prod_cat1,prod_cat2,prod_cat3
0,3,0.0,0.0
1,1,6.0,14.0


In [57]:
df_sample["prod_cat1"] = df_sample["prod_cat1"].astype("int").astype("str")
df_sample["prod_cat2"] = df_sample["prod_cat2"].astype("int").astype("str")
df_sample["prod_cat3"] = df_sample["prod_cat3"].astype("int").astype("str")

In [58]:
df_sample["prod_cat"] = df_sample["prod_cat1"] + "-" + df_sample["prod_cat2"] + "-" + df_sample["prod_cat3"]
df_sample.head(2)

,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,0,0,3-0-0
1,1,6,14,1-6-14


In [59]:
df_sample["prod_cat"].nunique()

21

In [60]:
df["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [71]:
df_q2 = df.loc[df["age_group"] == "26-35",]
df_q2 = df_q2[["user", "marital", "prod_cat1", "prod_cat2", "prod_cat3"]]
df_q2 = df_q2.reset_index(drop=True)
df_q2.head()

,user,marital,prod_cat1,prod_cat2,prod_cat3
0,3,0,1,2.0,NaN
1,5,1,8,NaN,NaN
2,5,1,5,11.0,NaN
3,5,1,8,NaN,NaN
4,5,1,8,NaN,NaN


In [72]:
df_q2 = df_q2.fillna(0)

In [73]:
df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("int").astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("int").astype("str")
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("int").astype("str")
df_q2["prod_cat"] = df_q2.loc[:, "prod_cat1":"prod_cat3"].apply(lambda x: x.str.cat(sep = "-"), axis = 1)

In [74]:
df_q2.head()

,user,marital,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,0,1,2,0,1-2-0
1,5,1,8,0,0,8-0-0
2,5,1,5,11,0,5-11-0
3,5,1,8,0,0,8-0-0
4,5,1,8,0,0,8-0-0


In [88]:
df_q2_agg = df_q2.groupby(["user", "marital"])["prod_cat"].nunique()
df_q2_agg = df_q2_agg.reset_index()
df_q2_agg.head(2)

,user,marital,prod_cat
0,3,0,18
1,5,1,43


In [89]:
df_q2_agg.groupby("marital")["prod_cat"].mean()

marital
0    41.663183
1    41.792336
Name: prod_cat, dtype: float64

## Q3 ANSWER

In [81]:
df_q3_user = df[["user", "gender", "age_group", "job", "city", "marital"]]
df_q3_user = df_q3_user.drop_duplicates().reset_index(drop=True)
df_q3_user.head(2)

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,A,0
1,2,M,55+,16,C,0


In [82]:
len(df_q3_user)

5891

In [83]:
df["user"].nunique()

5891

In [84]:
df.columns

Index(['user', 'prod', 'gender', 'age_group', 'job', 'city', 'marital',
       'prod_cat1', 'prod_cat2', 'prod_cat3', 'purchase'],
      dtype='object')

In [85]:
df_q3_agg1 = df.groupby("user")["prod"].nunique().reset_index()
df_q3_agg2 = df.groupby("user")["purchase"].sum().reset_index()
df_q3_agg1.head(2)

,user,prod
0,1,35
1,2,77


In [86]:
df_join = df_q3_user.merge(df_q3_agg1, on = "user")
df_join = df_join.merge(df_q3_agg2, on = "user")
df_join.head()

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472
2,3,M,26-35,15,A,0,29,341635
3,4,M,46-50,7,B,1,14,206468
4,5,M,26-35,20,A,1,106,821001


In [87]:
df_join["gender"] = df_join["gender"].replace({"M":1, "F":0})

SyntaxError: invalid syntax (2025799653.py, line 1)